In [26]:
import pandas as pd

df=pd.read_excel("MicroCenter_catagorized.xlsx").drop("Unnamed: 0",axis=1)
df.head()

,name,Is_Leasable,Category
0,SanDisk Clip Sport Go 32GB MP3 Player - Black,Leasable,MP3 Player
1,Inland 8GB MP3/MP4 Player - Black,Leasable,MP3 Player
2,SanDisk Clip Jam 8GB MP3 Player - Black,Leasable,MP3 Player
3,SanDisk Clip Sport Go 16GB MP3 Player - Black,Leasable,MP3 Player
4,SanDisk Clip Sport Plus 16GB MP3 Player - Blue,Leasable,MP3 Player


In [27]:
df.shape

(10599, 3)

In [28]:
df.isnull().sum()

name           1
Is_Leasable    0
Category       0
dtype: int64

In [29]:
df=df.dropna()

In [30]:
df.isnull().sum()

name           0
Is_Leasable    0
Category       0
dtype: int64

In [31]:
df.dtypes

name           object
Is_Leasable    object
Category       object
dtype: object

In [32]:
import re
import nltk
nltk.download('stopwords')
import string
string.punctuation

stopword = nltk.corpus.stopwords.words('english')

def to_lower_case(text):
    return text.lower()

def remove_punctuation(text):
    no_punct=[words for words in text if words not in string.punctuation]
    words_wo_punct=''.join(no_punct)
    return words_wo_punct

def tokenize(text):
    split=re.split("\W+",text) 
    return split

def remove_stopwords(text):
    text=[word for word in text if word not in stopword]
    return text


    

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\AyanSau\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [33]:
df=df.applymap(to_lower_case)
df=df.applymap(remove_punctuation)
df=df.applymap(tokenize)
df=df.applymap(remove_stopwords)
df=df.applymap(lambda x:" ".join(x))

In [34]:
df.head()

,name,Is_Leasable,Category
0,sandisk clip sport go 32gb mp3 player black,leasable,mp3 player
1,inland 8gb mp3mp4 player black,leasable,mp3 player
2,sandisk clip jam 8gb mp3 player black,leasable,mp3 player
3,sandisk clip sport go 16gb mp3 player black,leasable,mp3 player
4,sandisk clip sport plus 16gb mp3 player blue,leasable,mp3 player


In [35]:
T=max(df["name"].apply(lambda x:len(x.split())).tolist()) # max size of sentences
T

52

In [36]:
T=max(df["Category"].apply(lambda x:len(x.split())).tolist()) # max size of sentences
T

3

In [37]:
from transformers import BertTokenizer
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')
import torch
import numpy as np

def sen_to_vec(sentence):
    tokens=tokenizer.tokenize(sentence)
    tokens = ['[CLS]'] + tokens + ['[SEP]']
    T=52
    padded_tokens=tokens +['[PAD]' for _ in range(T-len(tokens))]
    attn_mask=[ 1 if token != '[PAD]' else 0 for token in padded_tokens  ]
    seg_ids=[0 for _ in range(len(padded_tokens))]
    sent_ids=tokenizer.convert_tokens_to_ids(padded_tokens)
    return np.array(sent_ids)

def sen_to_vec_cat(sentence):
    tokens=tokenizer.tokenize(sentence)
    tokens = ['[CLS]'] + tokens + ['[SEP]']
    T=3
    padded_tokens=tokens +['[PAD]' for _ in range(T-len(tokens))]
    attn_mask=[ 1 if token != '[PAD]' else 0 for token in padded_tokens  ]
    seg_ids=[0 for _ in range(len(padded_tokens))]
    sent_ids=tokenizer.convert_tokens_to_ids(padded_tokens)
    return np.array(sent_ids)

In [38]:
df["sent_vectors"]=df["name"].apply(sen_to_vec)
df["cat_vectors"]=df["Category"].apply(sen_to_vec_cat)
df.head()

,name,Is_Leasable,Category,sent_vectors,cat_vectors
0,sandisk clip sport go 32gb mp3 player black,leasable,mp3 player,"[101, 5472, 20573, 12528, 4368, 2175, 3590, 18...","[101, 23378, 2447, 102]"
1,inland 8gb mp3mp4 player black,leasable,mp3 player,"[101, 9514, 1022, 18259, 23378, 8737, 2549, 24...","[101, 23378, 2447, 102]"
2,sandisk clip jam 8gb mp3 player black,leasable,mp3 player,"[101, 5472, 20573, 12528, 9389, 1022, 18259, 2...","[101, 23378, 2447, 102]"
3,sandisk clip sport go 16gb mp3 player black,leasable,mp3 player,"[101, 5472, 20573, 12528, 4368, 2175, 2385, 18...","[101, 23378, 2447, 102]"
4,sandisk clip sport plus 16gb mp3 player blue,leasable,mp3 player,"[101, 5472, 20573, 12528, 4368, 4606, 2385, 18...","[101, 23378, 2447, 102]"


In [39]:
df_final_vec=df[["sent_vectors","cat_vectors"]]
df_final_vec.head()

,sent_vectors,cat_vectors
0,"[101, 5472, 20573, 12528, 4368, 2175, 3590, 18...","[101, 23378, 2447, 102]"
1,"[101, 9514, 1022, 18259, 23378, 8737, 2549, 24...","[101, 23378, 2447, 102]"
2,"[101, 5472, 20573, 12528, 9389, 1022, 18259, 2...","[101, 23378, 2447, 102]"
3,"[101, 5472, 20573, 12528, 4368, 2175, 2385, 18...","[101, 23378, 2447, 102]"
4,"[101, 5472, 20573, 12528, 4368, 4606, 2385, 18...","[101, 23378, 2447, 102]"


In [40]:
# taking only 10 rows to test

df_test=df.head(n=500)

In [41]:
# finding all possible combinations
from itertools import combinations
 
# comb = combinations(df["Name-Link"].tolist(), 2)

comb = combinations(df_test["name"].tolist(), 2) 
comb_list=list(map(lambda x:list(x),comb))

comb_list


[['sandisk clip sport go 32gb mp3 player black',
  'inland 8gb mp3mp4 player black'],
 ['sandisk clip sport go 32gb mp3 player black',
  'sandisk clip jam 8gb mp3 player black'],
 ['sandisk clip sport go 32gb mp3 player black',
  'sandisk clip sport go 16gb mp3 player black'],
 ['sandisk clip sport go 32gb mp3 player black',
  'sandisk clip sport plus 16gb mp3 player blue'],
 ['sandisk clip sport go 32gb mp3 player black',
  'sandisk clip sport go 16gb mp3 player black'],
 ['sandisk clip sport go 32gb mp3 player black',
  'sandisk clip sport go 32gb mp3 player blue'],
 ['sandisk clip sport go 32gb mp3 player black',
  'dgl group gpsmobile car mount'],
 ['sandisk clip sport go 32gb mp3 player black',
  'midland xtalker t20 walkie talkie 4 pack'],
 ['sandisk clip sport go 32gb mp3 player black', 'garmin rv 780 traffic'],
 ['sandisk clip sport go 32gb mp3 player black',
  'midland xtalker 38 mile twoway radio kit 2pack'],
 ['sandisk clip sport go 32gb mp3 player black',
  'midland frs avp

In [42]:
len(comb_list)

124750

In [43]:
def vectorize(sentences):
    return list(map(sen_to_vec,sentences))

df_to_find_sim=pd.DataFrame()
df_to_find_sim["Combinations"]=comb_list
df_to_find_sim["Arrays"]=df_to_find_sim["Combinations"].apply(vectorize)

In [44]:
df_to_find_sim.head()

,Combinations,Arrays
0,"[sandisk clip sport go 32gb mp3 player black, ...","[[101, 5472, 20573, 12528, 4368, 2175, 3590, 1..."
1,"[sandisk clip sport go 32gb mp3 player black, ...","[[101, 5472, 20573, 12528, 4368, 2175, 3590, 1..."
2,"[sandisk clip sport go 32gb mp3 player black, ...","[[101, 5472, 20573, 12528, 4368, 2175, 3590, 1..."
3,"[sandisk clip sport go 32gb mp3 player black, ...","[[101, 5472, 20573, 12528, 4368, 2175, 3590, 1..."
4,"[sandisk clip sport go 32gb mp3 player black, ...","[[101, 5472, 20573, 12528, 4368, 2175, 3590, 1..."


In [45]:
import math
from scipy import spatial

#(v1 dot v2)/{||v1||*||v2||}
def cosine_similarity(array):
    v1=list(array[0])
    v2=list(array[1])
    
    return 1 - spatial.distance.cosine(v1, v2)


df_to_find_sim["similarity"]=df_to_find_sim["Arrays"].apply(cosine_similarity)

In [64]:
# testing

#df_to_find_sim.to_excel("df_to_find_sim.xlsx")


df_to_find=df_to_find_sim[df_to_find_sim["similarity"]<=0.15]
list_prod= [item for sublist in df_to_find["Combinations"].tolist() for item in sublist]
len(set(list_prod))

df_prod=pd.DataFrame()
df_prod["name"]=list(set(list_prod))


In [65]:
df_prod

,name
0,sonos five wireless speaker streaming music white
1,mackie crx series 3inch 2 channel stereo multi...
2,inland proht 100 manual projection screen
3,bose sound link micro portable bluetooth speak...
4,sonos sonos one gen 2 black
...,...
233,accessory power gogroove wireless pc computer ...
234,lg sn6y 31 channel high res audio sound bar dt...
235,fitbit luxe fitness tracker lunar white
236,kensington martfit monitor stand 21 screens black


In [66]:
df_prod.to_excel("df_prod_15.xlsx")

In [56]:
def get_prod_1(string):
    return string[0]

def get_prod_2(string):
    return string[1]

df_to_find_sim["Product_1"]=df_to_find_sim["Combinations"].apply(get_prod_1)
df_to_find_sim["Product_2"]=df_to_find_sim["Combinations"].apply(get_prod_2)

df_to_find_sim.to_excel("Product_Similarity.xlsx")

In [57]:
df_to_find_sim.head()

,Combinations,Arrays,similarity,Product_1,Product_2
0,"[sandisk clip sport go 32gb mp3 player black, ...","[[101, 5472, 20573, 12528, 4368, 2175, 3590, 1...",0.414691,sandisk clip sport go 32gb mp3 player black,inland 8gb mp3mp4 player black
1,"[sandisk clip sport go 32gb mp3 player black, ...","[[101, 5472, 20573, 12528, 4368, 2175, 3590, 1...",0.772884,sandisk clip sport go 32gb mp3 player black,sandisk clip jam 8gb mp3 player black
2,"[sandisk clip sport go 32gb mp3 player black, ...","[[101, 5472, 20573, 12528, 4368, 2175, 3590, 1...",0.999530,sandisk clip sport go 32gb mp3 player black,sandisk clip sport go 16gb mp3 player black
3,"[sandisk clip sport go 32gb mp3 player black, ...","[[101, 5472, 20573, 12528, 4368, 2175, 3590, 1...",0.997587,sandisk clip sport go 32gb mp3 player black,sandisk clip sport plus 16gb mp3 player blue
4,"[sandisk clip sport go 32gb mp3 player black, ...","[[101, 5472, 20573, 12528, 4368, 2175, 3590, 1...",0.999530,sandisk clip sport go 32gb mp3 player black,sandisk clip sport go 16gb mp3 player black
